<a href="https://colab.research.google.com/github/ucsd-cse-spis-2023/ucsd-cse-spis-2023-peter-ritvik-project/blob/main/Categorical_output.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! gdown 1xE_aw7Ys5TGFuFBSRc2PwSVX1QuHQKP5     #download the csv file from drive

Downloading...
From: https://drive.google.com/uc?id=1xE_aw7Ys5TGFuFBSRc2PwSVX1QuHQKP5
To: /content/tripadvisor_hotel_reviews.csv
100% 15.0M/15.0M [00:00<00:00, 165MB/s]


In [2]:
pip install pandas    #install pandas librry

In [3]:
import numpy as np    #importing numpy and pandas
import pandas as pd
from tensorflow import keras    #importing keras
from tensorflow.keras.preprocessing.text import Tokenizer     #importing tokenizer to tokenize the words in the reviews data set
from tensorflow.keras.models import Sequential  #builds a sequential neural network layer by layer
from tensorflow.keras.layers import Dense #forms a layer of the neural network
from tensorflow.keras.layers import Softmax  #converts the outputted vecot values to probabilities
from keras.callbacks import EarlyStopping    #stops the model early if it detects overfitting
from tensorflow.keras.layers import Dropout #drops a certain ammount of neurons from the layer
from tensorflow.keras.regularizers import l2   #penalyzes large weights in the trainign data

data = pd.read_csv('tripadvisor_hotel_reviews.csv') #reads the csv file

reviews = data['Review'].tolist()      # adding reviews and ratings to list
ratings = data['Rating'].tolist()


training_reviews = data['Review'][:16000].tolist() #first 160000 reviews are training data
testing_reviews = data['Review'][16000:].tolist()  #last 4000 reviews are testing data

# Subtract 1 so each review's index is the review value - 1
training_ratings = data['Rating'][:16000].to_numpy() - 1  #first 160000 ratings are training data
testing_ratings = data['Rating'][16000:].to_numpy() - 1   #last 4000 are testing data


tokenizer = Tokenizer()
tokenizer.fit_on_texts(reviews)   #tokenizer analyzes the list 'reviews'
vocab_size = len(tokenizer.word_index) + 1    #calculates the total number of unique words in the data set


encoded_training_reviews = tokenizer.texts_to_matrix(training_reviews, mode='binary') #the neural netwrok needs numerical data so we encode the reviews to a binary matrix
encoded_testing_reviews = tokenizer.texts_to_matrix(testing_reviews, mode='binary')



categorical_training_ratings = keras.utils.to_categorical(training_ratings) #makes the training and testing ratings catorigical data
categorical_testing_ratings = keras.utils.to_categorical(testing_ratings)

In [ ]:
model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(vocab_size,), kernel_regularizer=l2(0.03))) #first layer of neural netwrok, 1024 neurons, all tokens inputed, regularized by multiplyer
model.add(Dense(256, activation='relu', input_shape=(1024,)))   #hidden layer of neural network with 256 neurons
model.add(Dropout(0.3))                                         # drops 30% of the neurons so thta genralized connections can be made (prevents overfitting)
model.add(Dense(128, activation='relu', input_shape=(256,)))    #hiden layer of neural netowrk with 128 neurons
model.add(Dense(64, activation='relu', input_shape=(128,)))     # hidden layer of neural network with 64 neurons
model.add(Dense(5, activation='softmax'))  #outputs probabilities into 5 classes

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])    #optimized data, calculates catagorical loss in accuracy, measures accuracy

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)   #stops the model from going through all epochs after minimal difference in the validation loss is detected 3 times
model.fit(encoded_training_reviews, categorical_training_ratings, validation_split=0.2,callbacks=[early_stopping], epochs=50, batch_size=64)    #tests 20% of the predicted data against testing data each time to see if overfitting is happening

In [ ]:
model.evaluate(encoded_testing_reviews, categorical_testing_ratings) #checks the loss and validation accuracy

In [ ]:
y_pred = model.predict(encoded_testing_reviews).argmax(axis=1) #find the index with the highest probability prediction along the axis of predicitons
y_test = categorical_testing_ratings.argmax(axis=1) #finds the testing data in the class with the highest probability

In [ ]:
import sklearn
from sklearn import metrics #metrics from sklearn to set up confiusion matrix

matrix = metrics.confusion_matrix(y_test, y_pred, normalize='all')
matrix_display = metrics.ConfusionMatrixDisplay(matrix)
matrix_display.plot() #displays confusion matrix showing how far predictions strayed aaway from actual ratings

In [ ]:
good_reviews = [input("Enter a positive hotel review --> ")]  #user inputs both a good and bad review
bad_reviews = [input("Enter a negative hotel review --> ")]

encoded_good_reviews = tokenizer.texts_to_matrix(good_reviews, mode='binary')
predicted_good_ratings = model.predict(encoded_good_reviews)    #user inputed good review is encoded as binary matrix

encoded_bad_reviews = tokenizer.texts_to_matrix(bad_reviews, mode='binary')   #user inputed bad review is encoded as binary matrix
predicted_bad_ratings = model.predict(encoded_bad_reviews)


testing_encoded_ratings = np.array(testing_ratings) / 5.0   #testing ratings are converted from 1-5 to 0-1
testing_encoded_reviews = tokenizer.texts_to_matrix(testing_reviews, mode='binary')  #testing reviews are converted to binary

predicted_good_ratings_list = predicted_good_ratings.flatten().tolist() #matrix is flatened and made into a list
predicted_good_rating = 0 #place holder initial values of 0 are ste before the code enters the for loop
max_good_probability = 0
for i in range(len(predicted_good_ratings_list)):   #for values in the range from 0 to the end of the list
    if predicted_good_ratings_list[i] > max_good_probability:   #if any i value is greater than the current max probability
      predicted_good_rating = i+1   #the i value is increased by 1 and saved as the new max probability
      max_good_probability = predicted_good_ratings_list[i]
print("good review prediction: ", predicted_good_rating, "stars")

predicted_bad_ratings_list = predicted_bad_ratings.flatten().tolist()
predicted_bad_rating = 0
max_bad_probability = 0
for i in range(len(predicted_bad_ratings_list)):
    if predicted_bad_ratings_list[i] > max_bad_probability:
      predicted_bad_rating = i+1
      max_bad_probability = predicted_bad_ratings_list[i]
print("bad review prediction: ",predicted_bad_rating , "stars")
